In [ ]:
# Install necessary libraries for QLoRA, PEFT, and Streamlit
!pip install -q streamlit transformers peft accelerate bitsandbytes torch pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 113.5 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
from pyngrok import ngrok
from google.colab import drive
import time
from datetime import datetime
import json

# --- A. Hugging Face Login (Required for Llama 2) ---
# Run this and paste your Hugging Face Access Token when prompted.
notebook_login()

# --- B. Ngrok Authentication (Required for Public Link) ---
# 🚨 REPLACE 'YOUR_AUTH_TOKEN' with your actual Ngrok token
NGROK_TOKEN = "####    YOUR NGROK TOKEN    ####"
ngrok.set_auth_token(NGROK_TOKEN)
print("✅ Ngrok authentication set.")

# --- C. Mount Google Drive (Required for model path) ---
drive.mount('/content/drive')
print("✅ Google Drive mounted.")

✅ Ngrok authentication set.
Mounted at /content/drive
✅ Google Drive mounted.


In [ ]:
%%writefile app.py
# Maternal Health AI Assistant - Full Code with Human Evaluation (Final Corrected Version)

import streamlit as st
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import time
from datetime import datetime
import json
import os

# =============================================================================
# CONFIGURATION
# =============================================================================

# CRITICAL PATH: Ensure this matches your mounted Google Drive path
BEST_MODEL_PATH = "/content/drive/MyDrive/maternal_health_project/models/llama2-maternal-health-qlora"
BASE_MODEL = "meta-llama/Llama-2-7b-chat-hf"

st.set_page_config(
    page_title="Maternal Health AI Assistant",
    page_icon=" ",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for Professional UI (Included here for completeness)
st.markdown("""
<style>
    /* 1. Typography and Main Color */
    .main-header { font-size: 2.8rem; color: #E91E63; text-align: center; margin-top: -20px;}
    .subtitle { text-align: center; color: #666; margin-bottom: 2rem; font-size: 1.1rem; }
    .emergency-box { background-color: #ffcccc; border-left: 5px solid #D32F2F; padding: 1rem; margin: 1rem 0; border-radius: 8px; box-shadow: 2px 2px 5px rgba(0, 0, 0, 0.1); }
    .chat-container { height: 60vh; overflow-y: auto; padding-right: 15px; border: 1px solid #ddd; border-radius: 10px; padding: 10px; }

    /* Custom Chat Message Styles */
    .stChatMessage [data-testid="stChatMessageContent"]:first-child { background-color: #F8BBD0; color: #333333; border-radius: 20px 20px 20px 5px; padding: 10px 15px; box-shadow: 1px 1px 3px rgba(0, 0, 0, 0.05); }
    .stChatMessage:nth-child(even) [data-testid="stChatMessageContent"]:first-child { background-color: #E3F2FD; border-radius: 20px 20px 5px 20px; color: #333333; }
    div.stButton > button:first-child { border-color: #E91E63; color: #E91E63; width: 100%; }
    div.stButton > button:first-child:hover { background-color: #FFCDD2; }

</style>
""", unsafe_allow_html=True)


# =============================================================================
# INITIALIZATION & LOGGING
# =============================================================================

# Initialize session state variables
if "messages" not in st.session_state:
    st.session_state.messages = []
if "human_eval_log" not in st.session_state:
    st.session_state.human_eval_log = []
if "conversation_id" not in st.session_state or len(st.session_state.messages) == 0:
    # Use a unique ID for each full conversation session
    st.session_state.conversation_id = datetime.now().strftime("%Y%m%d_%H%M%S")

# Define logging function for evaluation submission
def log_feedback(message_index, prompt, response, rel_val, emp_val, text_val):
    st.session_state.human_eval_log.append({
        "timestamp": datetime.now().isoformat(),
        "conversation_id": st.session_state.conversation_id,
        "message_id": message_index,
        "user_query": prompt,
        "model_response_snippet": response[:100] + "...",
        "score_relevance": rel_val,
        "score_empathy": emp_val,
        "feedback_text": text_val
    })
    st.toast("Feedback recorded! Thank you.", icon='📝')

    # CRITICAL: Set the evaluation flag to True to hide the form on next rerun
    st.session_state.messages[message_index]["evaluated"] = True


# =============================================================================
# LOAD MODEL (Cached for performance and QLoRA fix)
# =============================================================================

@st.cache_resource
def load_maternal_health_model():
    """Load the QLoRA model: Base Model + Adapter"""

    with st.spinner("Loading AI model components... This may take a minute..."):

        # Load Tokenizer from BASE MODEL
        tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "right"

        # Quantization Config for T4 GPU
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )

        # Load Base Model with 4-bit config and float16
        base_model = AutoModelForCausalLM.from_pretrained(
            BASE_MODEL,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True,
            torch_dtype=torch.float16
        )

        # Load & Merge Adapter
        model = PeftModel.from_pretrained(base_model, BEST_MODEL_PATH)
        model.eval()

    return model, tokenizer

# =============================================================================
# EVALUATION RENDERING FUNCTION (PERSISTENCE LOGIC)
# =============================================================================

def render_evaluation_form(index, prompt, response):
    """Renders the persistent evaluation form for a specific message."""

    # Use a unique key for the form based on the message index (ID)
    with st.form(key=f"feedback_form_{index}"):
        st.subheader("📝 Evaluate Empathy")

        # Metrics (1 = Poor/Inaccurate, 5 = Excellent/Highly Supportive)
        relevance = st.slider(
            "1. Medical Accuracy/Relevance (1-5)",
            min_value=1, max_value=5, value=3, key=f"rel_{index}",
            help="Was the information correct and relevant to the user's situation?"
        )

        empathy = st.slider(
            "2. Emotional Empathy & Tone (1-5)",
            min_value=1, max_value=5, value=3, key=f"emp_{index}",
            help="Did the response sound supportive, understanding, and non-judgmental?"
        )

        qualitative = st.text_area(
            "3. Qualitative Feedback (Optional)",
            placeholder="E.g., 'Sounded too robotic' or 'Very helpful and kind tone.'",
            key=f"qual_{index}"
        )

        # Submission button with logging callback
        st.form_submit_button(
            "Submit Evaluation",
            on_click=log_feedback,
            # Pass the message index, prompt, and response
            args=(index, prompt, response, relevance, empathy, qualitative)
        )


# =============================================================================
# SAFETY & GENERATION LOGIC
# =============================================================================

EMERGENCY_KEYWORDS = ["bleeding", "severe pain", "can't breathe", "chest pain", "decreased movement", "seizure"]
RESTRICTED_TOPICS = ["abortion procedures", "home abortion", "terminate pregnancy at home"]

def check_safety_protocols(query):
    query_lower = query.lower()
    if any(k in query_lower for k in EMERGENCY_KEYWORDS):
        return True, "⚠️ **URGENT MEDICAL EMERGENCY**: This sounds like a critical situation. Please **call 911** or seek immediate emergency medical care. This AI assistant cannot replace trained medical professionals."
    if any(t in query_lower for t in RESTRICTED_TOPICS):
        return True, "I cannot provide information on that topic. Please consult with a qualified healthcare provider for guidance on sensitive medical matters."
    return False, None

def add_disclaimer(response):
    disclaimer = "\n\n---\n*📋 Disclaimer: This information is for educational purposes only and does not replace professional medical advice. Always consult your healthcare provider.*"
    return response + disclaimer

def generate_response(query, model, tokenizer, chat_history_str=""):

    is_safe, warning_msg = check_safety_protocols(query)
    if is_safe:
        return warning_msg

    # Construct Prompt (Llama-2 Chat Format)
    system_prompt = """You are a highly knowledgeable, empathetic maternal health assistant.
    Provide accurate, evidence-based information from trusted sources like WHO, CDC, ACOG.
    Be supportive, encouraging, and empathetic. Always prioritize safety."""

    if chat_history_str:
        prompt = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\nPrevious Context:\n{chat_history_str}\n\nCurrent Question: {query} [/INST]"
    else:
        prompt = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{query} [/INST]"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=350,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,
        )

    generated_tokens = outputs[0][inputs.input_ids.shape[1]:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return add_disclaimer(response.strip())

# =============================================================================
# STREAMLIT UI LAYOUT & MAIN LOGIC
# =============================================================================

st.markdown('<h1 class="main-header"> Maternal Health AI Assistant</h1>', unsafe_allow_html=True)
st.markdown('<p class="subtitle">Evidence-based support for your pregnancy and postpartum journey</p>', unsafe_allow_html=True)

# Sidebar (Static Info and Log Viewer)
with st.sidebar:
    st.header("ℹ️ Evaluation Session")
    st.info(f"Session ID: **{st.session_state.conversation_id}**\n\nAdapter Path: `{BEST_MODEL_PATH}`")

    st.markdown("---")

    st.markdown('<div class="emergency-box">', unsafe_allow_html=True)
    st.header("🚨 Emergency Guide")
    st.markdown("Call 911 for severe symptoms.")
    st.markdown('</div>', unsafe_allow_html=True)

    st.markdown("---")

    st.header("📋 Evaluation Log")
    st.write(f"Logged Turns: **{len(st.session_state.human_eval_log)}**")
    if st.button("Download Log (JSON)"):
        json_string = json.dumps(st.session_state.human_eval_log, indent=2)
        st.download_button(
            label="Download Data",
            data=json_string,
            file_name=f'evaluation_log_{st.session_state.conversation_id}.json',
            mime='application/json'
        )

    if st.button("🗑️ Reset Conversation & Log"):
        st.session_state.messages = []
        st.session_state.human_eval_log = []
        st.rerun()

# Load Model
try:
    model, tokenizer = load_maternal_health_model()
    st.sidebar.success("✅ AI Assistant is Ready!")
except Exception as e:
    st.error(f"❌ Error loading model: {e}. Check path and Mount.")
    st.stop()


# Main Chat Area (Display history and persistent evaluation form)
st.markdown('<div class="chat-container">', unsafe_allow_html=True)
for i, message in enumerate(st.session_state.messages):
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

        # NEW LOGIC: Render Form Persistently
        if message["role"] == "assistant" and message.get("evaluated", False) == False:
            render_evaluation_form(
                i,
                message.get("raw_prompt", "N/A"),
                message.get("raw_response", "N/A")
            )

st.markdown('</div>', unsafe_allow_html=True)


# Chat Input and Generation Logic
if prompt := st.chat_input("Ask about prenatal nutrition, safe medications, or common symptoms..."):

    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    # Generate Response
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):

            # --- FIX FOR NameError: history_context ---
            history_context = ""

            # Prepare chat history context
            recent_msgs = st.session_state.messages[-7:-1]
            for msg in recent_msgs:
                # FIX FOR SyntaxError: unterminated string literal
                role = "User" if msg['role'] == 'user' else "Assistant"
                history_context += f"{role}: {msg['content']}\n"

            # The full response object is needed for logging
            full_response = generate_response(prompt, model, tokenizer, history_context)

            # Display the generated response
            st.markdown(full_response)

            # Store data necessary for the persistent evaluation form
            message_index = len(st.session_state.messages)

            st.session_state.messages.append({
                "role": "assistant",
                "content": full_response,
                "raw_response": full_response,
                "raw_prompt": prompt,
                "evaluated": False
            })

            # Immediately render the form for the newest message
            render_evaluation_form(
                message_index,
                prompt,
                full_response
            )

    st.rerun()

Writing app.py


In [ ]:
# Start the Streamlit server in the background
!streamlit run app.py &>/dev/null&

# Wait for the Streamlit server to start
import time
print("Attempting to tunnel Streamlit app...")
time.sleep(5)

# Create the Ngrok tunnel
try:
    tunnel = ngrok.connect(8501)

    print("-" * 50)
    print("✨ Streamlit App is LIVE! Click the link below: ✨")
    print(f"PUBLIC LINK: {tunnel.public_url}")
    print("-" * 50)

    # Keep the cell alive until stopped manually
    print("\nTunnel active. Press the **SQUARE STOP BUTTON** to terminate.")
    while True:
        time.sleep(1)

except KeyboardInterrupt:
    print("\nStopping Streamlit server and disconnecting Ngrok...")
    ngrok.disconnect(tunnel.public_url)

except Exception as e:
    print(f"❌ Failed to run Streamlit/Ngrok: {e}")

Attempting to tunnel Streamlit app...
--------------------------------------------------
✨ Streamlit App is LIVE! Click the link below: ✨
PUBLIC LINK: https://joltily-uncapitalistic-susan.ngrok-free.dev
--------------------------------------------------

Tunnel active. Press the **SQUARE STOP BUTTON** to terminate.

Stopping Streamlit server and disconnecting Ngrok...


PyngrokNgrokURLError: ngrok client exception, URLError: [Errno 111] Connection refused

In [ ]:
!ps aux | grep streamlit

root       10537  0.0  0.0   7376  3544 ?        S    03:30   0:00 /bin/bash -c ps aux | grep streamlit
root       10539  0.0  0.0   6484  2468 ?        S    03:30   0:00 grep streamlit
